In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, r2_score

data_dir = Path(".")

redfin_files = {
    "Peoria": "redfin_peoria_metro.csv",
    "Lima": "redfin_Lima_metro.csv",
    "Sheboygan": "redfin_sheboygan_metro.csv",
    "Duluth": "redfin_Duluth_metro.csv"
}

def load_and_clean_redfin(path, metro_name):
    df = pd.read_csv(path)

    # Drop the first disclaimer row from Redfin
    # It usually starts with "In accordance with local MLS rules..."
    df = df[~df["SALE TYPE"].astype(str).str.contains("In accordance", na=False)]

    # Basic column rename to simpler names
    df = df.rename(columns={
        "SOLD DATE": "sold_date",
        "PROPERTY TYPE": "property_type",
        "ADDRESS": "address",
        "CITY": "city",
        "STATE OR PROVINCE": "state",
        "ZIP OR POSTAL CODE": "zip",
        "PRICE": "closed_price",
        "BEDS": "beds",
        "BATHS": "baths",
        "SQUARE FEET": "sqft",
        "LOT SIZE": "lot_size",
        "YEAR BUILT": "year_built",
        "DAYS ON MARKET": "dom",
        "$/SQUARE FEET": "price_per_sqft_raw",
        "HOA/MONTH": "hoa_fee",
        "LATITUDE": "latitude",
        "LONGITUDE": "longitude"
    })

    # Keep only past sales
    df = df[df["SALE TYPE"].isin(["PAST SALE"])]

    # Strip and standardize zip
    df["zip"] = df["zip"].astype(str).str.extract(r"(\d{5})", expand=False)

    # Helper to clean numeric columns
    def clean_numeric(col):
        return (
            df[col]
            .astype(str)
            .str.replace(r"[,$]", "", regex=True)
            .str.replace(r"[^\d\.\-]", "", regex=True)  # remove text like "sq ft"
            .replace("", np.nan)
            .astype(float)
        )

    numeric_cols = ["closed_price", "beds", "baths", "sqft",
                    "year_built", "dom", "price_per_sqft_raw", "hoa_fee"]
    for col in numeric_cols:
        if col in df.columns:
            df[col] = clean_numeric(col)

    # Lot size is messy strings; keep a simple numeric extraction
    if "lot_size" in df.columns:
        df["lot_size"] = (
            df["lot_size"]
            .astype(str)
            .str.replace(",", "", regex=False)
            .str.extract(r"([\d\.]+)", expand=False)
            .replace("", np.nan)
            .astype(float)
        )

    # Drop rows without a price or sqft (cannot be used for model)
    df = df.dropna(subset=["closed_price", "sqft"])

    # Add metro label
    df["metro"] = metro_name

    return df

frames = []
for metro, filename in redfin_files.items():
    path = data_dir / filename
    frames.append(load_and_clean_redfin(path, metro))

all_props = pd.concat(frames, ignore_index=True)

print("Combined dataset shape:", all_props.shape)
print(all_props[["metro", "city", "state", "zip", "closed_price", "beds", "baths", "sqft"]].head())


Combined dataset shape: (1219, 28)
    metro            city state    zip  closed_price  beds  baths    sqft
0  Peoria          Peoria    IL  61615      176500.0   3.0    1.0  1897.0
1  Peoria          Morton    IL  61550      330000.0   4.0    2.5  2168.0
2  Peoria      Washington    IL  61571      115000.0   3.0    1.0   874.0
3  Peoria           Pekin    IL  61554      159900.0   3.0    2.5  1928.0
4  Peoria  Peoria Heights    IL  61616      105000.0   2.0    1.0   720.0


C:\Users\abhis\AppData\Local\Temp\ipykernel_32128\897857692.py:59: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace("", np.nan)
C:\Users\abhis\AppData\Local\Temp\ipykernel_32128\897857692.py:59: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace("", np.nan)
C:\Users\abhis\AppData\Local\Temp\ipykernel_32128\897857692.py:59: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the 

In [2]:
# 2.1 Load ZIP -> CBSA crosswalk
cross = pd.read_csv("zip_cbsa_crosswalk.csv")

# Standardize ZIP formats
cross["ZIP"] = cross["ZIP"].astype(str).str.zfill(5)
all_props["zip"] = all_props["zip"].astype(str).str.zfill(5)

# Merge ONLY ZIP and CBSA (NO CBSA_NAME in this file)
all_props = all_props.merge(
    cross[["ZIP", "CBSA"]],
    left_on="zip",
    right_on="ZIP",
    how="left"
).drop(columns=["ZIP"])

print("After ZIP->CBSA merge:")
print(all_props[["metro", "zip", "CBSA"]].head())


After ZIP->CBSA merge:
    metro    zip   CBSA
0  Peoria  61615  37900
1  Peoria  61550  37900
2  Peoria  61571  37900
3  Peoria  61554  37900
4  Peoria  61616  37900


In [3]:
# 2.2 Load Model-1 city scores (this has CBSA_NAME + all scoring dimensions)
metro_scores = pd.read_csv("FinalMasterData.csv")

metro_scores_trim = metro_scores[[
    "CBSA",
    "CBSA_NAME",
    "ES_score",
    "HG_score",
    "MG_score",
    "CL_score",
    "AF_score",
    "Final_City_Score"
]].drop_duplicates()

# Merge Model-1 scoring dimensions into property dataset
all_props = all_props.merge(
    metro_scores_trim,
    on="CBSA",
    how="left"
)

print("After merging Model-1 scores:")
print(all_props[["metro", "CBSA", "CBSA_NAME", "Final_City_Score"]].head())


After merging Model-1 scores:
    metro   CBSA   CBSA_NAME  Final_City_Score
0  Peoria  37900  Peoria, IL          0.595813
1  Peoria  37900  Peoria, IL          0.595813
2  Peoria  37900  Peoria, IL          0.595813
3  Peoria  37900  Peoria, IL          0.595813
4  Peoria  37900  Peoria, IL          0.595813


In [4]:
# Create a simple property age feature
current_year = 2025
all_props["age"] = current_year - all_props["year_built"]

# Basic null handling for model
for col in ["beds", "baths", "sqft", "dom", "lot_size", "hoa_fee", "age"]:
    if col in all_props.columns:
        all_props[col] = all_props[col].fillna(all_props[col].median())

# If any Model-1 scores are missing (for some properties), fill with metro-level median
score_cols = ["ES_score", "HG_score", "MG_score", "CL_score", "AF_score", "Final_City_Score"]
for col in score_cols:
    if col in all_props.columns:
        all_props[col] = all_props[col].fillna(all_props[col].median())

# Encode metro as categorical if you want the model to see it
all_props["metro_cat"] = all_props["metro"].astype("category").cat.codes

feature_cols = [
    "beds", "baths", "sqft",
    "lot_size", "hoa_fee", "age",
    "metro_cat"
]

# Add Model-1 features if present
for col in score_cols:
    if col in all_props.columns:
        feature_cols.append(col)

X = all_props[feature_cols].copy()
y = all_props["closed_price"].astype(float)

print("Feature columns:", feature_cols)
print("X shape:", X.shape, "y shape:", y.shape)







X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model = GradientBoostingRegressor(random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE (USD): {mae:,.0f}")
print(f"R²: {r2:.3f}")




Feature columns: ['beds', 'baths', 'sqft', 'lot_size', 'hoa_fee', 'age', 'metro_cat', 'ES_score', 'HG_score', 'MG_score', 'CL_score', 'AF_score', 'Final_City_Score']
X shape: (1562, 13) y shape: (1562,)
MAE (USD): 61,281
R²: 0.675


c:\Users\abhis\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [17]:
# Predict fair price for all rows (not just test)
all_props["predicted_fair_price"] = model.predict(X)

# Compute residuals on the test set to estimate noise
residuals = y_test - y_pred
sigma = residuals.std()



In [23]:
# Use a simple ±25% band around predicted fair price
BAND_PCT = 0.25   # 25%

all_props["fair_price_low_95"]  = all_props["predicted_fair_price"] * (1 - BAND_PCT)
all_props["fair_price_high_95"] = all_props["predicted_fair_price"] * (1 + BAND_PCT)




In [24]:
SAFETY_MARGIN_PCT = 0.05  # 5% discount

all_props["max_offer_price"] = all_props["predicted_fair_price"] * (1 - SAFETY_MARGIN_PCT)


In [34]:
all_props["relative_band_width"] = (
    (all_props["fair_price_high_95"] - all_props["fair_price_low_95"])
    / all_props["predicted_fair_price"].replace(0, np.nan)
)

UNCERTAINTY_THRESHOLD = 0.50  # 15% band width

all_props["review_flag"] = np.where(
    all_props["relative_band_width"] > UNCERTAINTY_THRESHOLD,
    "REVIEW_REQUIRED",
    "OK"
)


In [38]:
all_props = all_props.reset_index(drop=True)
all_props["property_id"] = all_props.index + 1

final_cols = [
    "property_id", "metro", "city", "state", "zip",
    "beds", "baths", "sqft", "lot_size", "age",
    "closed_price",
    "predicted_fair_price",
    "fair_price_low_95", "fair_price_high_95",
    "max_offer_price",
    "review_flag",
    "CBSA", "CBSA_NAME", "Final_City_Score"
]

df2 = all_props[final_cols]
df2.to_csv("Model2_AcquisitionPrices_combined.csv", index=False)

df2.head()


,property_id,metro,city,state,zip,beds,baths,sqft,lot_size,age,closed_price,predicted_fair_price,fair_price_low_95,fair_price_high_95,max_offer_price,review_flag,CBSA,CBSA_NAME,Final_City_Score
0,1,Peoria,Peoria,IL,61615,3.0,1.0,1897.0,9170.0,56.0,176500.0,186893.900008,140170.425006,233617.375010,177549.205008,REVIEW_REQUIRED,37900,"Peoria, IL",0.595813
1,2,Peoria,Morton,IL,61550,4.0,2.5,2168.0,13200.0,50.0,330000.0,264995.382823,198746.537117,331244.228529,251745.613682,OK,37900,"Peoria, IL",0.595813
2,3,Peoria,Washington,IL,61571,3.0,1.0,874.0,11600.0,77.0,115000.0,113478.896607,85109.172455,141848.620759,107804.951777,REVIEW_REQUIRED,37900,"Peoria, IL",0.595813
3,4,Peoria,Pekin,IL,61554,3.0,2.5,1928.0,17859.0,63.0,159900.0,225758.246762,169318.685071,282197.808452,214470.334424,OK,37900,"Peoria, IL",0.595813
4,5,Peoria,Peoria Heights,IL,61616,2.0,1.0,720.0,6098.0,75.0,105000.0,82298.079254,61723.559440,102872.599067,78183.175291,OK,37900,"Peoria, IL",0.595813


In [36]:
df2 = pd.read_csv("Model2_AcquisitionPrices_combined.csv")
df2.head()
df2.columns


Index(['property_id', 'metro', 'city', 'state', 'zip', 'closed_price',
       'predicted_fair_price', 'fair_price_low_95', 'fair_price_high_95',
       'max_offer_price', 'review_flag', 'CBSA', 'CBSA_NAME',
       'Final_City_Score'],
      dtype='object')